In [1]:
# Run in terminal or command prompt
# python3 -m spacy download en

import numpy as np
import pandas as pd
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet as wn
import re, nltk, spacy, gensim, string

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\driss\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# empty doc list
doc_complete = []

with open (".\\data\\stackoverflow_vacature_tekst2.csv", "r") as myfile:
    for line in myfile:
        doc_complete.append(line)

In [3]:
tech_words_set = set()

with open(".\\data\\top50000_tags_short2.csv", "r") as myfiletags:
    for line in myfiletags:
        #tech_words_set.add(line.replace('"',''))
        tech_words_set.add(line.rstrip())
        
#for word in tech_words_set:
#    print(word)


In [4]:
stop = set(stopwords.words(['english', 'german', 'dutch']))
#stop_de = set(stopwords.words('german'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    #stop_de_free = " ".join([i for i in stop_free.lower().split() if i not in stop_de])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    #print(normalized)
    #[print(i) for i in normalized.lower().split()]
    normalized2 = " ".join([i for i in normalized.lower().split() if i in tech_words_set])
    #print(normalized2)
    return normalized2
    ####return normalized

doc_clean = [clean(doc).split() for doc in doc_complete] 

print(doc_clean[:10])

[['build', 'python', 'php', 'java', 'go', 'stack'], ['design', 'c', 'javascript', 'sql', 'process', 'order', 'c', 'sql', 'server', 'javascript', 'angularjs', 'focus', 'typescript', 'angular', 'cloud', 'azure'], ['design', 'c', 'javascript', 'sql', 'process', 'order', 'c', 'sql', 'server', 'javascript', 'angularjs', 'focus', 'typescript', 'angular', 'cloud', 'azure'], ['build', 'python', 'php', 'java', 'go', 'stack'], ['express', 'express', 'api', 'stack', 'express', 'include', 'api', 'automation', 'java', 'cloud', 'web', 'angular', 'browser', 'mobile', 'count', 'background', 'web', 'angular', 'java', 'database', 'html', 'dynamic', 'web', 'web', 'design', 'angular', 'java', 'web', 'html', 'javascript', 'spring', 'design', 'testing', 'selenium', 'junit', 'jenkins', 'selenium'], ['python', 'time', 'service', 'interface', 'python', 'python', 'database', 'mysql', 'redis', 'mongodb', 'linux', 'window', 'server', 'web', 'html', 'jquery', 'angularjs', 'php', 'cakephp'], ['join', 'web', 'html5'

In [5]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(doc_clean, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:2])

['build python php java go stack', 'design c javascript sql process order c sql server javascript angularj focus typescript angular cloud azure']


In [6]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words=['english','german'],  # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{1,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [7]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  4.97809147374 %


In [5]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(doc_clean, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:2])

['build python php java go stack', 'design c javascript sql process order c sql server javascript angularj focus typescript angular cloud azure']


In [8]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=20,           # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=20, n_jobs=-1, n_topics=None, perp_tol=0.1,
             random_state=100, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)


In [9]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -304186.945921
Perplexity:  103.14626722
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 20,
 'n_jobs': -1,
 'n_topics': None,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [10]:
# Define Search Param
search_params = {'n_components': [5, 10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params,cv=5)

# Do the Grid Search
model.fit(data_vectorized)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_components': [5, 10, 15, 20, 25, 30], 'learning_decay': [0.5, 0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [11]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

print(best_lda_model.n_topics)


Best Model's Params:  {'learning_decay': 0.7, 'n_components': 5}
Best Log Likelihood Score:  -63216.282609
Model Perplexity:  100.478609059
None


In [13]:
# Create Document - Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)

# column names
topicnames = ['Topic' + str(i) for i in range(best_lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(doc_clean))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(100).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,dominant_topic
Doc0,0.03,0.03,0.03,0.88,0.03,3
Doc1,0.8,0.01,0.01,0.01,0.16,0
Doc2,0.8,0.01,0.01,0.01,0.16,0
Doc3,0.03,0.03,0.03,0.88,0.03,3
Doc4,0.76,0.01,0.01,0.01,0.22,0
Doc5,0.37,0.01,0.01,0.6,0.01,3
Doc6,0.97,0.01,0.01,0.01,0.01,0
Doc7,0.73,0.02,0.02,0.21,0.02,0
Doc8,0.03,0.03,0.15,0.76,0.03,3
Doc9,0.08,0.55,0.01,0.34,0.01,1


In [14]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

,Topic Num,Num Documents
0,2,690
1,4,638
2,0,619
3,3,491
4,1,437


In [15]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=            Freq  cluster  topics           x           y
topic                                                    
2      26.233474        1       1  142.059692   61.417110
4      21.671503        1       2   71.195557  160.965637
0      19.827476        1       3  137.864426  -60.704247
1      16.247004        1       4   12.991290  -41.025555
3      16.020543        1       5  -22.091438   75.654770, topic_info=     Category         Freq        Term        Total  loglift  logprob
term                                                                 
252   Default  1432.000000    security  1432.000000  30.0000  30.0000
129   Default  1185.000000  javascript  1185.000000  29.0000  29.0000
299   Default  2163.000000         web  2163.000000  28.0000  28.0000
43    Default  1592.000000       cloud  1592.000000  27.0000  27.0000
172   Default  1045.000000      mobile  1045.000000  26.0000  26.0000
94    Default   842.000000         get   842.000000  25.0000  25.0000
6     Default   454.000000     android   454.000000  24.0000  24.0000
128   Default  1505.000000        java  1505.000000  23.0000  23.0000
217   Default  1027.000000      python  1027.000000  22.0000  22.0000
296   Default   469.000000       video   469.000000  21.0000  21.0000
204   Default   436.000000         php   436.000000  20.0000  20.0000
84    Default   300.000000    facebook   300.000000  19.0000  19.0000
282   Default  1645.000000     testing  1645.000000  18.0000  18.0000
113   Default   327.000000        html   327.000000  17.0000  17.0000
64    Default  4117.000000      design  4117.000000  16.0000  16.0000
213   Default  1812.000000     process  1812.000000  15.0000  15.0000
271   Default   759.000000       stack   759.000000  14.0000  14.0000
22    Default   863.000000  automation   863.000000  13.0000  13.0000
69    Default   497.000000      docker   497.000000  12.0000  12.0000
7     Default   342.000000     angular   342.000000  11.0000  11.0000
151   Default   613.000000       linux   613.000000  10.0000  10.0000
266   Default   330.000000      spring   330.000000   9.0000   9.0000
230   Default   385.000000      report   385.000000   8.0000   8.0000
114   Default   218.000000       html5   218.000000   7.0000   7.0000
195   Default   217.000000      oracle   217.000000   6.0000   6.0000
32    Default   795.000000           c   795.000000   5.0000   5.0000
41    Default  1031.000000      client  1031.000000   4.0000   4.0000
285   Default  1166.000000        time  1166.000000   3.0000   3.0000
77    Default   177.000000       embed   177.000000   2.0000   2.0000
245   Default   232.000000       scala   232.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
94     Topic5   585.834037         get   842.512213   1.4679  -2.8872
145    Topic5   104.915608   kubernete   155.117849   1.4403  -4.6071
69     Topic5   300.300021      docker   497.045859   1.3274  -3.5554
98     Topic5   234.507098          go   399.433553   1.2987  -3.8027
217    Topic5   526.378247      python  1027.922212   1.1620  -2.9942
11     Topic5   104.995209     ansible   165.126127   1.3785  -4.6063
271    Topic5   385.663075       stack   759.097249   1.1541  -3.3053
180    Topic5   174.897994       mysql   310.149917   1.2584  -4.0960
211    Topic5   102.120144  postgresql   170.932282   1.3162  -4.6341
87     Topic5   141.278203        find   303.156186   1.0678  -4.3095
185    Topic5    36.190076       nginx    51.029320   1.4877  -5.6714
4      Topic5   147.986221   algorithm   332.669874   1.0213  -4.2631
135    Topic5   297.351091        join  1010.633955   0.6079  -3.5653
285    Topic5   304.554484        time  1166.271675   0.4886  -3.5414
32     Topic5   229.003159           c   795.225272   0.5864  -3.8265
63     Topic5   201.192607  deployment   676.931508   0.6180  -3.9560
14     Topic5   140.420589         api   394.434660   0.7985  -4.3156
196    Topic5   101.499198       order   274.415735

In [16]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames

# View
df_topic_keywords.head()

,3d,air,ajax,akka,algorithm,alignment,android,angular,angularj,angularjs,...,wcf,web,webdriver,webpack,window,wordpress,wpf,xamarin,xcode,xml
Topic0,11.003871,7.131387,53.193964,0.201421,7.029596,0.201138,0.920130,330.637430,121.030706,12.168002,...,32.702851,1535.139865,0.324438,109.166978,3.125109,31.195214,10.448527,33.195486,0.225614,51.929878
Topic1,0.201705,0.203492,0.200324,0.200217,7.604691,0.201814,453.474791,0.200867,0.200725,0.200052,...,0.200149,41.008767,0.201088,0.200870,151.597565,0.200922,0.201326,0.202119,22.171582,2.582843
Topic2,0.202022,4.271979,0.200492,0.200301,2.905347,59.193884,0.201828,0.202016,0.203410,0.200980,...,0.202696,299.881220,10.070044,0.200399,25.889869,0.200738,2.055484,0.201572,0.202094,5.918002
Topic3,74.390911,13.520169,0.203000,0.232710,147.697456,0.200977,0.201912,14.683483,0.203985,6.224235,...,1.693120,167.124332,0.203026,0.230028,46.914607,0.203107,5.094096,0.200287,0.200707,0.201532
Topic4,0.201490,1.872973,0.202221,35.165352,167.762911,0.202187,0.201340,2.276204,0.361174,0.206732,...,0.201184,147.845816,0.201405,0.201725,3.472851,0.200019,0.200567,0.200537,0.200003,58.367745


In [17]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,web,javascript,design,build,html,angular,mobile,service,sql,testing,stack,client,git,position,c
Topic 1,security,build,mobile,service,android,design,linux,facebook,server,c,automation,deployment,java,scripting,cloud
Topic 2,design,service,process,testing,architecture,performance,automation,position,time,key,video,build,report,web,database
Topic 3,get,python,stack,php,service,build,time,docker,join,go,c,process,deployment,position,mysql
Topic 4,cloud,java,service,design,client,build,database,architecture,spring,sql,python,performance,model,scala,oracle
